In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/yoga-posture-dataset/Poses.json
/kaggle/input/yoga-posture-dataset/Camatkarasana/File50.png
/kaggle/input/yoga-posture-dataset/Camatkarasana/File44.png
/kaggle/input/yoga-posture-dataset/Camatkarasana/File24.png
/kaggle/input/yoga-posture-dataset/Camatkarasana/File42.png
/kaggle/input/yoga-posture-dataset/Camatkarasana/File52.png
/kaggle/input/yoga-posture-dataset/Camatkarasana/File9.png
/kaggle/input/yoga-posture-dataset/Camatkarasana/File53.png
/kaggle/input/yoga-posture-dataset/Camatkarasana/File26.png
/kaggle/input/yoga-posture-dataset/Camatkarasana/File45.png
/kaggle/input/yoga-posture-dataset/Camatkarasana/File5.png
/kaggle/input/yoga-posture-dataset/Camatkarasana/File4.png
/kaggle/input/yoga-posture-dataset/Camatkarasana/File36.png
/kaggle/input/yoga-posture-dataset/Camatkarasana/File46.png
/kaggle/input/yoga-posture-dataset/Camatkarasana/File13.png
/kaggle/input/yoga-posture-dataset/Camatkarasana/File12.png
/kaggle/input/yoga-posture-dataset/Camatkarasana/File59.j

## Loading the CSV file containing extracted keypoints

In [ ]:

keypoints_df = pd.read_csv('/kaggle/input/keypoints-of-poses/yoga_keypoints.csv')
print(keypoints_df.head())

          0         1         2         3         4         5         6  \
0  0.379307  0.268986 -0.249039  0.403164  0.258503 -0.292665  0.405915   
1  0.542991  0.000873 -0.056974  0.560612  0.000000 -0.086612  0.561551   
2  0.582899  0.296683 -0.056250  0.609204  0.301483 -0.080810  0.610430   
3  0.364177  0.314397 -0.133954  0.384789  0.293991 -0.167073  0.389159   
4  0.497106  0.075594 -0.259878  0.524652  0.087406 -0.293495  0.527161   

          7         8         9  ...        90        91        92   93  \
0  0.260801 -0.292517  0.408872  ...  0.139663  0.943306  0.643350  1.0   
1  0.003118 -0.086581  0.562749  ...  0.990231  0.891121  0.301815  0.0   
2  0.308129 -0.080818  0.611666  ...  0.870198  0.907586 -0.086578  0.0   
3  0.294353 -0.166945  0.393661  ...  0.139931  0.972488  0.429078  1.0   
4  0.093818 -0.293323  0.530002  ...  0.125889  0.982878  0.225679  1.0   

         94        95   96        97        98         label  
0  1.000000 -0.332617  0.0  0.94101

## Importing Libraries 

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

## Adding Labels and splitting the data

In [5]:
X = keypoints_df.drop(columns=["label"]).values.astype(float)  # Keypoints (99 values per sample)
y = keypoints_df["label"].values  # Pose labels

# Encode the labels as integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # Encode the labels as integers

# Split the data into training, validation, and test sets (80% train, 10% validation, 10% test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Normalize the keypoints (optional but helps in training)
X_train = X_train / np.linalg.norm(X_train, axis=1, keepdims=True)
X_val = X_val / np.linalg.norm(X_val, axis=1, keepdims=True)
X_test = X_test / np.linalg.norm(X_test, axis=1, keepdims=True)


## Checking the size of the test and train set

In [6]:
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (2106, 99)
y_train shape: (2106,)
X_val shape: (263, 99)
y_val shape: (263,)
X_test shape: (264, 99)
y_test shape: (264,)


## Multi Layer Perceptron

In [7]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(99,)),  # 99 keypoints
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')  # Unique pose classes
])

# Compile the model
model.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Fit the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=30, batch_size=32)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.0444 - loss: 3.8353 - val_accuracy: 0.0456 - val_loss: 3.7164
Epoch 2/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0696 - loss: 3.6525 - val_accuracy: 0.1141 - val_loss: 3.3074
Epoch 3/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1240 - loss: 3.2522 - val_accuracy: 0.2243 - val_loss: 2.8097
Epoch 4/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1830 - loss: 2.8553 - val_accuracy: 0.3156 - val_loss: 2.4609
Epoch 5/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2209 - loss: 2.5533 - val_accuracy: 0.4106 - val_loss: 2.2861
Epoch 6/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2991 - loss: 2.3879 - val_accuracy: 0.3954 - val_loss: 2.1023
Epoch 7/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3173 - loss: 2.2934 - val_accuracy: 0.4867 - val_loss: 1.9795
Epoch 8/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3456 - loss: 2.1637 - val_accuracy: 0.4487 - val_loss:

## Saving the model

In [19]:
model.save('yoga_pose_mlp_model.h5')

## Checking the accuracy and loss

In [37]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7683 - loss: 0.9459 
Test Loss: 1.0274115800857544
Test Accuracy: 0.7462121248245239


## Testing of model 

In [38]:
# Make predictions
predictions = model.predict(X_test)

# Convert predicted probabilities to labels
predicted_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))

# Display the predictions
for i in range(5):
    print(f"True label: {label_encoder.inverse_transform([y_test[i]])[0]}, Predicted label: {predicted_labels[i]}")

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
True label: Ardha Chandrasana, Predicted label: Ardha Chandrasana
True label: Vasisthasana, Predicted label: Adho Mukha Vrksasana
True label: Salamba Bhujangasana, Predicted label: Urdhva Mukha Svsnssana
True label: Parsvottanasana, Predicted label: Parsvottanasana
True label: Urdhva Mukha Svsnssana, Predicted label: Urdhva Mukha Svsnssana


## Setting the ideal poses 

In [ ]:
# Initialize a dictionary to store ideal poses
ideal_poses_1 = {}

# Keypoints of interest (indices to keep)
keypoints_of_interest = [0, 1, 13, 14, 12, 10, 9, 11, 23, 24, 26, 28, 25, 27]

# Iterate over each unique label (pose name)
for label in keypoints_df['label'].unique():
    # Filter rows corresponding to the current pose
    pose_data = keypoints_df[keypoints_df['label'] == label]
    
    # Extract and average the keypoints of interest for the pose
    pose_keypoints = []
    for _, row in pose_data.iterrows():
        keypoints = []
        for i in keypoints_of_interest:
            keypoints.append(row.iloc[i])  # Extract only the keypoints of interest
        pose_keypoints.append(keypoints)
    
    # Calculate the mean pose for this label across all samples
    mean_keypoints = np.mean(pose_keypoints, axis=0).tolist()
    
    # Store the averaged keypoints in the ideal_poses dictionary
    ideal_poses_1[label] = mean_keypoints

# Optional: Convert to a DataFrame for better visualization
ideal_poses_df = pd.DataFrame.from_dict(ideal_poses_1, orient='index', columns=[f'KP_{i}' for i in keypoints_of_interest])


# Save to CSV if needed
ideal_poses_df.to_csv("ideal_poses_filtered.csv", index=True)


## Comparing ideal pose and test pose

In [ ]:

keypoint_names = {
    0: "Head",
    1: "Neck",
    9: "Left Wrist",
    10: "Right Wrist",
    11: "Left Elbow",
    12: "Right Elbow",
    13: "Left Shoulder",
    14: "Right Shoulder",
    23: "Left Hip",
    24: "Right Hip",
    25: "Left Knee",
    26: "Right Knee",
    27: "Left Ankle",
    28: "Right Ankle"
}
'''
# Function to calculate the angle formed by three keypoints
def calculate_angle(a, b, c):
   
    ba = np.array(a) - np.array(b)
    bc = np.array(c) - np.array(b)
    cos_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.degrees(np.arccos(cos_angle))
    return angle
'''
# Function to compare poses using Euclidean distance
def compare_poses(test_pose_keypoints, ideal_pose_keypoints):
    
    deviations = []
    
    
    # Loop over corresponding keypoints in test and ideal poses
    for test_point, ideal_point in zip(test_pose_keypoints, ideal_pose_keypoints):
        test_x, test_y = test_point
        ideal_x, ideal_y = ideal_point
        
        # Measure the deviation in terms of Euclidean distance
        distance = np.sqrt((test_x - ideal_x)**2 + (test_y - ideal_y)**2)
        deviations.append(distance)

        
    return deviations

# Function to compare test pose with ideal pose and calculate average deviation
def compare_test_pose(test_pose, ideal_poses_1, keypoints_of_interest):

    # Access ideal pose keypoints by label (ideal poses are now lists of keypoints)
    label = test_pose['label']  # Assuming the test pose has a 'label' to reference the ideal pose
    ideal_pose_keypoints = ideal_poses_1[label]  # Ideal pose corresponding to the label
    
    # Extract the test pose keypoints for the specified indices
    test_pose_keypoints = []
    for i in keypoints_of_interest:
        x = test_pose.iloc[i]  # Extract x-coordinate
        y = test_pose.iloc[i + 1]  # Extract y-coordinate
        test_pose_keypoints.append((x, y))  # Store (x, y) as a tuple
    
    # Compare with the ideal pose
    deviations = compare_poses(test_pose_keypoints, ideal_pose_keypoints)
    
    # Return the average deviation for the pose
    average_deviation = np.mean(deviations)

    feedback = []
    for i, (test_point, ideal_point) in enumerate(zip(test_pose_keypoints, ideal_pose_keypoints)):
        test_x, test_y = test_point
        ideal_x, ideal_y = ideal_point
        
        # Directional feedback based on x and y deviations
        if test_x < ideal_x:
            x_feedback = "Move right"
        elif test_x > ideal_x:
            x_feedback = "Move left"
        else:
            x_feedback = "Correct horizontal position"
        
        if test_y < ideal_y:
            y_feedback = "Move down"
        elif test_y > ideal_y:
            y_feedback = "Move up"
        else:
            y_feedback = "Correct vertical position"

        keypoint_name = keypoint_names.get(keypoints_of_interest[i], f"Keypoint {i + 1}")  # Default to "Keypoint X" if name is not found
        
        feedback.append(f"{keypoint_name}: {x_feedback}, {y_feedback}")
        
    return average_deviation, feedback

# Example of ideal poses (in your format, with flattened lists for keypoints)


# Convert the flattened list into tuples of coordinates
for pose_name, keypoints in ideal_poses_1.items():
    ideal_poses_1[pose_name] = [(keypoints[i], keypoints[i+1]) for i in range(0, len(keypoints), 2)]

# Example of comparing a test pose
keypoints_of_interest = [0, 1, 13, 14, 12, 10, 9, 11, 23, 24, 26, 28, 25, 27]  # Selected keypoints
test_pose = keypoints_df.iloc[0]  # Select a test pose (row 0)

# Compare the test pose with ideal poses and calculate the deviation
deviation, feedback= compare_test_pose(test_pose, ideal_poses_1, keypoints_of_interest)
print(f"Average deviation from ideal pose: {deviation}")
print("Feedback on how to improve the pose:")
for i, suggestion in enumerate(feedback):
    print(suggestion)


Average deviation from ideal pose: 0.4763629839124
Feedback on how to improve the pose:
Head: Move right, Move up
Neck: Move left, Move down
Left Shoulder: Move right, Move down
Right Shoulder: Move right, Move up
Right Elbow: Move left, Move down
Right Wrist: Move left, Move down
Left Wrist: Move left, Move down
